In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
from utils.preprocessing import get_gray_images

In [2]:
testing_dir = '../data/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data'
training_dir_base = f'../data/C-NMC_Leukemia/training_data/'
fold_0_all_path = 'fold_0/all'

In [3]:
### The code for this lives in notebooks.utils.preprocessing
get_gray_images(training_dir_base+fold_0_all_path)[0]

<tf.Tensor: shape=(450, 450, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>